In [1]:
import pandas as pd
import numpy as np #mathematical variables
from random import normalvariate #used for stockhastic process
import plotly.express as px #python library for complex plot

from cadCAD.configuration.utils import config_sim
from cadCAD.configuration import Experiment
from cadCAD.engine import ExecutionContext, Executor

### 1. State Variables

The states we are interested in, their state variables and their initial values are:

* The __atmosphere's CO2 concentration__ in parts per million (ppm): `co2`, initial value 400
* The __earth's surface temperature__ in Kelvin (K): `temperature`, initial value 290
    
<!--**Create a dictionary and define the above state variables and their initial values:**-->

In [2]:
initial_state = {
    'co2' : 400,
    'temperature' : 290
}

In [3]:
initial_state

{'co2': 400, 'temperature': 290}

**The system parameters we need to define are:**

* The sun radiation: `sun_radiation` with value `1361`
* A constant representing the relationship between temperature and radiation: `temperature_constant` with value `1e-4`
* A constant representing CO2 impact on the radiation balance via the greenhouse effect: `co2_reflectance_factor` with value `1e-3`
* A unit conversion constant that relates how much gigatons of CO2 we need to have an additional part per million unit in the atmosphere's concentration: `co2_gigatons_to_ppm` with value `1.2e-1`
* The standard deviation for the stochastic process generating the yearly CO2 concentration: `co2_stdev` with value `40` ppm
* A constant representing how much heat dissipitates into space: `heat_dissipation_constant` with value `2075`

**There are two parameters which we want to sweep, which are:**

* A parameter which represents the annual CO2 emissions in units of billion tons, which is the `co2_annual_emissions`. Let's sweep three values for it: `0`, `40` and `80`. The first value simulates a scenario where we stop all emissions at once, while using `40` means no additional emissions beyond what we already emmit every year, and `80` means that we are going to double our emissions.
* The `year_of_the_wakening`, which is the number of years that must pass before we set the `co2_annual_emissions` to zero. Let's sweep four values for it: `0`, `10`, `50` and `100`.

<!--**Create a dictionary and define the above parameters and their initial values:**-->

In [4]:
system_params = {
    'sun_radiation' : [1361],
    'temperature_constant' : [1e-4],
    'co2_reflectance_factor' : [1e-3],
    'co2_gigatons_to_ppm' : [1.2e-1],
    'co2_stdev' : [40],
    'heat_dissipation_constant' : [2075],
    'co2_annual_emissions' : [40, 80, 40, 80, 40, 80, 40, 80],
    'year_of_the_wakening' : [0,0,10,10,50,50,100,100]
}

1e10 is scientific notation in Python.
It means:
👉 1 × 10ⁱ⁰ = 10,000,000,000

__1x10xx10 means the same thing:__
__👉 1 × (10 to the power of 10) = 10,000,000,000__

✅ So what does assert do here?
assert checks if both sides are equal.

If they are, nothing happens — the code continues.

If they're not equal, Python throws an AssertionError.

🧠 So in simple terms:
“Hey Python, make sure 1e10 and __1 * 10**10 are the same number.”__

And they are, so the code passes without errors.

In [5]:
#defining e
assert 1e10 == 1*10**10

### 3. Policy Functions

In [6]:
def p_co2_emissions(params, substep, state_history, previous_state):
    #Parameters & variables
    mean = params['co2_annual_emissions']
    std = params['co2_stdev']
    conversion_factor = params['co2_gigatons_to_ppm']
    t_w = params['year_of_the_wakening']
    t = previous_state['timestep']
    
    #Logic
    if t > t_w:
        mean = 0
    else:
        mean = mean
    value = normalvariate(mean, std) * conversion_factor
    
    #Output
    return {'add_co2' : value}

>__🎯 What does this function do?__

It decides __how much extra CO2 goes into the air__ every year, based on how much humans are polluting and how random the weather (or other factors) are.

>__🧩 Let’s break it down simply:__

Imagine the __Earth is like a big bathtub,__ and __CO2 is water__ going into the tub.
The `p_co2_emission` function helps decide:

“How much CO2 (water) should we pour into the bathtub this year?”

>__🔧 What pieces does it use?__

`mean:` How much CO2 we usually add per year (like turning the faucet on)

`std:` A number that makes the CO2 amount a little random each year (because life is messy!)

`conversion_factor:` Helps us turn "tons of CO2" into "how much it changes the air"

`t_w:` The year when humans wake up and decide to stop polluting

`t:` What year it is right now (like keeping track of time in the game)

>__🧠 What is the logic?__

1. If it's __after the wake-up year__ `(t > t_w)`, people __stop adding CO2,__ so we set `mean = 0`.

2. If it’s __before that,__ people keep polluting as normal.

3. We then use a random generator (`normalvariate`) to say:

>>“This year, we’ll add about `mean` CO2, but maybe a little more or less depending on randomness.”

4. Then we multiply by a conversion factor so we can say:

>>“This much CO2 turns into this much ppm (air concentration).”

>__🔚 What does it give back?__

It returns a dictionary like this:

{'add_co2': some number}

That number is __how much new CO2 gets added__ this year.

>__🧪 Example:__

Let’s say:

-  It's year 5

-  People will wake up at year 50

-  They currently emit 80 units of CO2 per year

The function might randomly return something like:

{'add_co2': 8.7}

But after year 50, it will always return:

{'add_co2': 0.0}

Because humans finally __stopped polluting__ 🌍

In [7]:
def p_sun_radiation (params, substep, state_history, previous_state):
    #Parameters & variables
    g = params['temperature_constant']
    a = params['sun_radiation']
    
    #Logic
    temp_change = g * a
    
    #Output
    return {'add_temperature' : temp_change}

In [8]:
def p_earth_cooling (params, substep, state_history, previous_state):
    #Parameters & variables
    g = params['temperature_constant']
    K = params['heat_dissipation_constant']
    T = previous_state['temperature']
    
    #Logic
    temp_change = -(g * K * (T/300)**4)
    
    #Output
    return {'add_temperature': temp_change}

In [9]:
def p_greenhouse_effect(params, substep, state_history, previous_state):
    # Parameters & variables
    g = params['temperature_constant']
    K = params['heat_dissipation_constant']
    beta = params['co2_reflectance_factor']
    T = previous_state['temperature']
    CO2 = previous_state['co2']
    
    # Logic
    alpha = (1 - np.exp(-beta * CO2))
    temp_change = g * alpha * K * (T / 300) ** 4
    
    # Output
    return {'add_temperature': temp_change}

### 4. State Update Functions

In [10]:
def s_co2 (params, substep, state_history, previous_state, policy_input):
    #Parameters & variables
    current_co2 = previous_state['co2']
    co2_change = policy_input['add_co2']
    
    #Logic
    new_co2 = max(current_co2 + co2_change, 0)
    
    #Output
    return ('co2', new_co2)

In [11]:
def s_temperature (params, substep, state_history, previous_state, policy_input):
    #Parametes & variables
    current_temp = previous_state['temperature']
    temp_change = policy_input['add_temperature']
    
    #Logic
    new_temp = max(current_temp + temp_change, 0)
    
    #Output
    return ('temperature', new_temp)

### 5. Partial State Update Blocks

In [12]:
partial_state_update_blocks = [
    {
        'label': 'Temperature dynamics', # Useful metadata to describe our partial state update blocks
        'policies': {
            'sun_radiation': p_sun_radiation,
            'earth_cooling': p_earth_cooling,
            'greenhouse_effect': p_greenhouse_effect
        },
        'variables': {
            'temperature': s_temperature
            
        }
    },
    {
        'label': 'CO2 dynamics', # Useful metadata to describe our partial state update blocks
        'policies': {
            'co2_emissions': p_co2_emissions
        },
        'variables': {
            'co2': s_co2
        }
        
    }
]


### 6. Configuration

In [13]:
MONTE_CARLO_RUNS = 50
SIMULATION_TIMESTEPS = 100

sim_config = config_sim(
    {
        'N': MONTE_CARLO_RUNS,
        'T': range(SIMULATION_TIMESTEPS),
        'M': system_params,
    }
)

from cadCAD import configs
del configs[:] # Clear any prior configs

experiment = Experiment()
experiment.append_configs(
    sim_configs=sim_config,
    initial_state=initial_state,
    partial_state_update_blocks=partial_state_update_blocks
)

### 7. Execution

In [14]:
exec_context = ExecutionContext()
run = Executor(exec_context=exec_context, configs=experiment.configs)

(system_events, tensor_field, sessions) = run.execute()


                  ___________    ____
  ________ __ ___/ / ____/   |  / __ \
 / ___/ __` / __  / /   / /| | / / / /
/ /__/ /_/ / /_/ / /___/ ___ |/ /_/ /
\___/\__,_/\__,_/\____/_/  |_/_____/
by cadCAD

cadCAD Version: 0.5.3
Execution Mode: local_proc
Simulation Dimensions:
Entire Simulation: (Models, Unique Timesteps, Params, Total Runs, Sub-States) = (1, 100, 8, 400, 2)
     Simulation 0: (Timesteps, Params, Runs, Sub-States) = (100, 8, 400, 2)


Initializing configurations:   0%|          | 0/400 [00:00<?, ?it/s]

Execution Method: parallelize_simulations


Exception in thread Thread-6:
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\threading.py", line 980, in _bootstrap_inner
    self.run()
  File "C:\ProgramData\Anaconda3\lib\threading.py", line 917, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\dozbilenler\AppData\Roaming\Python\Python39\site-packages\multiprocess\pool.py", line 519, in _handle_workers
    cls._wait_for_updates(current_sentinels, change_notifier)
  File "C:\Users\dozbilenler\AppData\Roaming\Python\Python39\site-packages\multiprocess\pool.py", line 499, in _wait_for_updates
    wait(sentinels, timeout=timeout)
  File "C:\Users\dozbilenler\AppData\Roaming\Python\Python39\site-packages\multiprocess\connection.py", line 882, in wait
    ready_handles = _exhaustive_wait(waithandle_to_obj.keys(), timeout)
  File "C:\Users\dozbilenler\AppData\Roaming\Python\Python39\site-packages\multiprocess\connection.py", line 814, in _exhaustive_wait
    res = _winapi.WaitForMultipleObjects

KeyboardInterrupt: 

In [ ]:
# Get system events and attribute index
df = (pd.DataFrame(system_events)
      .assign(years = lambda df: df.timestep)
      .assign(temperature_celsius = lambda df: df.temperature - 273)
      .query('timestep > 0')
     )

#clean substeps
first_ind = (df.substep == 0) & (df.timestep == 0)
last_ind = df.substep == max(df.substep)
inds_to_drop = (first_ind | last_ind)
df = df.loc[inds_to_drop].drop(columns = ['substep'])

#Attribute parameters to each row
df = df.assign(**experiment.configs[0].sim_config['M'])
for i, (_, n_df) in enumerate(df.groupby(['simulation', 'subset', 'run'])):
    df.loc[n_df.index] = n_df.assign(**experiment.configs[i].sim_config['M'])

1. __Turn simulation results into a table and add useful columns__

`df = (pd.DataFrame(system_events)
      .assign(years = lambda df: df.timestep)
      .assign(temperature_celsius = lambda df: df.temperature - 273)
      .query('timestep > 0')
     )`
     
👉 We take the simulation results (`system_events`) and turn them into a table called `df`.

We add a column called `years`, which is just the `timestep` (like how many years have passed).

We also create a new column called `temperature_celsius`, by changing the temperature from __Kelvin__ (used in science) to __Celsius__ (what people use in daily life) by subtracting 273.

Then we say: __“Only keep the rows where `timestep` is more than 0”__ — because we don’t care about the very first moment (it's like skipping the first second of a video).

2. __Remove extra rows from the start and end of each update step__

`first_ind = (df.substep == 0) & (df.timestep == 0)
last_ind = df.substep == max(df.substep)
inds_to_drop = (first_ind | last_ind)
df = df.loc[inds_to_drop].drop(columns = ['substep'])`

👉 cadCAD simulations split each step into smaller pieces called __substeps__.

- We figure out which rows are at the __very beginning__ or __very end__ of these updates (we don't need those — they're a bit messy).

- We __keep only those__ (`inds_to_drop`) and remove the column called substep because we don’t need it anymore.

3. __Add the parameters that were used for each simulation__


`df = df.assign(**configs[0].sim_config['M'])`

👉 We look at the first simulation setup (`configs[0]`) and __add the input parameters__ (like sun radiation, emissions, etc.) as new columns in our table so we know which numbers were used for each run.

4. __Fix this for each simulation run__

`for i, (_, n_df) in enumerate(df.groupby(['simulation', 'subset', 'run'])):
    df.loc[n_df.index] = n_df.assign(**configs[i].sim_config['M'])`
    
👉 We’re looping through __each simulation run__ (like round 1, round 2, etc.) and making sure each group of rows gets __their exact parameter values__. This is important because we ran many scenarios, and we want to match the results to the correct inputs.

🧠 __Summary for a 10-Year-Old__

"We’re taking a big list of results from our experiment, turning it into a table, and cleaning it up by:

- Adding new info like Celsius temperature and years,

- Throwing away some messy rows we don’t need,

- And finally attaching a label to each row that tells us what settings were used when that result was made — kind of like writing names on our LEGO builds so we remember what we were testing.”

## What-if Matrix

What-if-question | Type of experiment | Variables / parameters | Values / Ranges to be tested
- | - | - | -
How will the __Earth's average temperature__ develop over the next 100 years, if we keep CO2 emissions __unchanged__ at today’s annual emission levels vs. a __doubling__ of today’s emission levels? | Parameter Sweep + Monte Carlo runs | co2_annual_emissions | 40 and 80 Gigatons
How will the __rate of annual temperature change__ develop over the next 100 years if we keep CO2 emissions __unchanged__ at today’s annual emission levels vs. a __doubling__ of today’s emission levels? | Parameter Sweep + Monte Carlo runs | co2_annual_emissions | 40 and 80 Gigatons
How will the __rate of annual temperature change__ develop over the next 100 years if we are able to reduce annual CO2 emissions to __zero__ after a given number of years? | Parameter Sweep + Monte Carlo runs | year_of_the_wakening | 0, 10, 50 and 100 years
How will the __Earth's average temperature__ develop over the next 100 years if we are able to reduce annual CO2 emissions to __zero__ after a given number of years? | Parameter Sweep + Monte Carlo runs | year_of_the_wakening | 0, 10, 50 and 100 years

## System Analysis

### Analysis 1: How will the Earth's average temperature develop over the next 100 years, if we keep CO2 emissions unchanged at today’s annual emission levels vs. a doubling of today’s emission levels?

In [ ]:
fig_df = df.query('year_of_the_wakening == 100')

fig = px.scatter(
    fig_df,
    x=fig_df.years,
    y=fig_df.temperature_celsius,
    color=fig_df.co2_annual_emissions.astype(str),
    opacity=0.1,
    trendline="lowess",
    labels={'color': 'Yearly CO2 emissions (Gt)'}
)

fig.show()